# **Weather Agent**

In [ ]:
# Install required libraries for weather agent

!pip install requests geopy env_canada nest_asyncio


In [ ]:
# Import libraries and define helper function
import requests
from geopy.geocoders import Nominatim
import xml.etree.ElementTree as ET
from IPython.display import Image, display
from google.colab import userdata
import time
import asyncio
from env_canada import ECWeather, ECAirQuality
import nest_asyncio

# Apply nest_asyncio for Colab
nest_asyncio.apply()

print("Libraries imported and helper function defined!")

In [ ]:
# User configuration
USER_ADDRESS = "931 College St, Toronto, ON M6J 2E7"  # Enter Your address
MAX_WALK_DISTANCE_KM = 5
MAX_BIKE_DISTANCE_KM = 10

print(USER_ADDRESS)

In [ ]:
# Step 1: Geocode the address

def geocode_address(address):
    """Geocode an address to latitude, longitude, and city using Nominatim."""
    print("=== Step1: Geocoding Address ===")
    print(f"Attempting to geocode: {address}")
    geolocator = Nominatim(user_agent="weather_risk_agent_unique_123")
    time.sleep(1)  # Respect Nominatim rate limits
    try:
        location = geolocator.geocode(address, timeout=10)
        if not location:
            print(f"Error: Could not find coordinates for {address}")
            return None, None, None
        lat, lon = location.latitude, location.longitude
        # Get city via reverse geocoding
        city = geolocator.reverse((lat, lon), language="en", timeout=10).raw.get("address", {}).get("city", "unknown").lower()
        print(f"Success: Geocoded {address} to (lat: {lat}, lon: {lon}, city: {city})")
        return lat, lon, city
    except Exception as e:
        print(f"Error: Geocoding failed with message: {str(e)}")
        return None, None, None

# Run immediately
user_lat, user_lon, user_city = geocode_address(USER_ADDRESS)


In [ ]:
# Step 2: Get weather data from Environment Canada

async def get_weather(lat, lon):
    """Fetch current weather from Environment Canada using env_canada."""
    print("=== Step 2: Fetching Weather Data ===")
    print(f"Querying Environment Canada for weather at (lat: {lat}, lon: {lon})")
    try:
        ec_weather = ECWeather(coordinates=(lat, lon))
        await ec_weather.update()
        condition = ec_weather.conditions.get("condition", {}).get("value", "").lower()
        temp = ec_weather.conditions.get("temperature", {}).get("value")
        wind = ec_weather.conditions.get("wind_speed", {}).get("value")
        wind_ms = float(wind) / 3.6 if wind else 0  # Convert km/h to m/s
        weather_data = {
            "main": condition,
            "temp": float(temp) if temp else None,
            "wind": wind_ms
        }
        print(f"Success: Weather data retrieved:")
        print(f"  - Condition: {weather_data['main']}")
        print(f"  - Temperature: {weather_data['temp']}°C")
        print(f"  - Wind speed: {weather_data['wind']:.1f} m/s")
        return weather_data
    except Exception as e:
        print(f"Error: Weather API failed with message: {str(e)}")
        return None

# Run immediately
if user_lat is None or user_lon is None:
    print("=== Step 2: Error: Cannot fetch weather without valid coordinates. ===")
    weather = None
else:
    weather = asyncio.get_event_loop().run_until_complete(get_weather(user_lat, user_lon))


In [ ]:
# Step 3: Get air quality (AQHI) from Google Air

def get_air_quality(lat, lon, api_key):
    """Fetch air quality data from Google Maps Air Quality API."""
    print("=== Step 3: Fetching Air Quality ===")
    print(f"Querying Google Maps API at (lat: {lat}, lon: {lon})")
    try:
        url = "https://airquality.googleapis.com/v1/currentConditions:lookup"
        headers = {"Content-Type": "application/json"}
        payload = {
            "location": {"latitude": lat, "longitude": lon},
            "extraComputations": ["HEALTH_RECOMMENDATIONS", "POLLUTANT_CONCENTRATION"]
        }
        response = requests.post(url, json=payload, headers=headers, params={"key": api_key}, timeout=30)
        response.raise_for_status()

        data = response.json()
        aqi = data.get("indexes", [{}])[0].get("aqi", 0)
        category = data.get("indexes", [{}])[0].get("category", "Unknown")
        # Combine generalPopulation and athletes recommendations
        health_recs = data.get("healthRecommendations", {})
        general_rec = health_recs.get("generalPopulation", "No recommendation available")
        athletes_rec = health_recs.get("athletes", "No recommendation available")
        recommendation = f"General: {general_rec}\nAthletes: {athletes_rec}"

        # Find PM2.5 specifically
        pm25 = 0
        for pollutant in data.get("pollutants", []):
            if pollutant.get("code") == "pm25":
                pm25 = pollutant.get("concentration", {}).get("value", 0)
                break
        pm25_note = "" if pm25 > 0 else " (estimated: AQI * 0.3)"
        if pm25 == 0 and aqi > 0:
            pm25 = min(15, max(5, aqi * 0.3))  # Fallback estimate

        result = f"AQI={aqi}, {category}, PM2.5={pm25} µg/m³{pm25_note}\nRecommendation:\n{recommendation}"
        print(f"Success: {result}")
        return aqi, category, pm25, recommendation
    except Exception as e:
        print(f"Error: Request failed: {str(e)}")
        print("Success: Data unavailable")
        return 0, "Unknown", 0, "No recommendation available"

# Run immediately
if user_lat is None or user_lon is None:
    print("=== Step 3: Error: Cannot fetch air quality without valid coordinates ===")
    aqi, category, pm25, recommendation = 0, "Unknown", 0, "No recommendation available"
else:
    GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY') # Load API key from secrets
    if GOOGLE_API_KEY is None:
        print("Error: GOOGLE_API_KEY not found in Colab Secrets.")
        aqi, category, pm25, recommendation = 0, "Unknown", 0, "No recommendation available"
    else:
        aqi, category, pm25, recommendation = get_air_quality(user_lat, user_lon, GOOGLE_API_KEY)

In [ ]:
# Step 4: Fetch active weather warnings from Environment Canada

def get_weather_warnings(city):
    """Fetch weather warnings from Environment Canada RSS feed."""
    print("=== Step 4: Fetching Weather Warnings ===")
    print(f"Querying Environment Canada for warnings in {city}")
    try:
        # Map city to RSS feed
        city_to_rss = {
            "montreal": "qc-147_e.xml",
            "toronto": "on-118_e.xml",
            "vancouver": "bc-84_e.xml",
            "unknown": "canada_e.xml"
        }
        rss_code = city_to_rss.get(city.lower(), "canada_e.xml")
        url = f"https://weather.gc.ca/rss/warning/{rss_code}"
        response = requests.get(url, timeout=30)
        response.raise_for_status()

        root = ET.fromstring(response.text)
        warnings = []
        for entry in root.findall(".//{http://www.w3.org/2005/Atom}entry"):
            title = entry.find("{http://www.w3.org/2005/Atom}title").text
            if city.lower() != "unknown" and city.title() in title:
                warnings.append(title)
            elif city.lower() == "unknown":
                warnings.append(title)

        print(f"Success: warnings={warnings or 'No warnings'}")
        return warnings
    except Exception as e:
        print(f"Error: Request failed: {str(e)}")
        return []

# Run immediately
if user_city is None:
    print("=== Step 4: Error: Cannot fetch warnings without valid city ===")
    warnings = []
else:
    warnings = get_weather_warnings(user_city)



In [ ]:
# new Step 5: Find nearby parks and bike trail Google Places API

# import requests # Already imported
from google.colab import userdata # Import userdata if not already imported


def get_parks_and_bike_routes(lat, lon, city, api_key, walk_distance_km, bike_distance_km):
    """Fetch nearby parks and bike routes with Google Places and Distance Matrix APIs."""
    print("=== Step 5: Fetching Parks and Bike Routes ===")
    print(f"Searching within {walk_distance_km} km for parks and {bike_distance_km} km for bike routes near {city} (lat: {lat}, lon: {lon})")
    try:
        # Fetch parks
        parks = []
        walk_distance_m = walk_distance_km * 1000
        url = f"https://maps.googleapis.com/maps/api/place/nearbysearch/json?location={lat},{lon}&radius={walk_distance_m}&type=park&key={api_key}"
        response = requests.get(url, timeout=30)
        response.raise_for_status()
        data = response.json()
        for result in data.get("results", [])[:2]:
            name = result.get("name", "Unknown Park")
            park_lat = result["geometry"]["location"]["lat"]
            park_lon = result["geometry"]["location"]["lng"]
            rating = result.get("rating", "N/A")
            # Get walking distance
            distance_url = f"https://maps.googleapis.com/maps/api/distancematrix/json?origins={lat},{lon}&destinations={park_lat},{park_lon}&mode=walking&key={api_key}"
            distance_response = requests.get(distance_url, timeout=30)
            distance_response.raise_for_status()
            distance_data = distance_response.json()
            distance_m = distance_data["rows"][0]["elements"][0]["distance"]["value"]  # Meters
            distance_km = distance_m / 1000
            if distance_km <= walk_distance_km:
                parks.append({"name": name, "lat": park_lat, "lon": park_lon, "distance": distance_km, "rating": rating})

        # Fetch bike routes
        bike_routes = []
        bike_distance_m = bike_distance_km * 1000
        url = f"https://maps.googleapis.com/maps/api/place/nearbysearch/json?location={lat},{lon}&radius={bike_distance_m}&type=park&keyword=cycling|trail|bike path|bicycles&key={api_key}"
        response = requests.get(url, timeout=30)
        response.raise_for_status()
        data = response.json()
        for result in data.get("results", [])[:5]:
            name = result.get("name", "Unknown Route")
            route_lat = result["geometry"]["location"]["lat"]
            route_lon = result["geometry"]["location"]["lng"]
            rating = result.get("rating", "N/A")
            # Get biking distance
            distance_url = f"https://maps.googleapis.com/maps/api/distancematrix/json?origins={lat},{lon}&destinations={route_lat},{route_lon}&mode=bicycling&key={api_key}"
            distance_response = requests.get(distance_url, timeout=30)
            distance_response.raise_for_status()
            distance_data = distance_response.json()
            distance_m = distance_data["rows"][0]["elements"][0]["distance"]["value"]  # Meters
            distance_km = distance_m / 1000
            if distance_km <= bike_distance_km:
                bike_routes.append({"name": name, "lat": route_lat, "lon": route_lon, "distance": distance_km, "rating": rating})

        # Output results (no sorting, use API order)
        print(f"Success: Found {len(parks)} parks and {len(bike_routes)} bike routes:")
        for park in parks:
            print(f"  - {park['name']} ({park['distance']:.1f} km, walk, Rating: {park['rating']}/5)")
        for route in bike_routes:
            print(f"  - {route['name']} ({route['distance']:.1f} km, bike, Rating: {route['rating']}/5)")
        return parks, bike_routes
    except Exception as e:
        print(f"Error: Request failed: {str(e)}")
        print("Success: No parks or bike routes found")
        return [], []

# Run immediately
if user_lat is None or user_lon is None:
    print("=== Step 5: Error: Cannot fetch parks or bike routes without valid coordinates ===")
    parks, bike_routes = [], []
else:
    GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY') # Load API key from secrets
    if GOOGLE_API_KEY is None:
        print("Error: GOOGLE_API_KEY not found in Colab Secrets.")
        parks, bike_routes = [], []
    else:
        parks, bike_routes = get_parks_and_bike_routes(user_lat, user_lon, user_city, GOOGLE_API_KEY, MAX_WALK_DISTANCE_KM, MAX_BIKE_DISTANCE_KM)


# from IPython.display import Image, display # Already imported

def generate_static_map(lat, lon, parks, bike_routes, api_key):
    try:
        map_url = f"https://maps.googleapis.com/maps/api/staticmap?center={lat},{lon}&zoom=12&size=600x400"
        for park in parks:
            map_url += f"&markers=color:yellow|label:W|{park['lat']},{park['lon']}"
        for route in bike_routes:
            map_url += f"&markers=color:red|label:B|{route['lat']},{route['lon']}"
        map_url += f"&key={api_key}"

        print("Success: Static map generated")
        return Image(url=map_url)
    except Exception as e:
        print(f"Error: Map generation failed: {str(e)}")
        print("Debug: No map due to error.")
        print("=== Step: Result: No map ===")
        return None

# Run map
if user_lat is None or user_lon is None or parks is None or bike_routes is None:
     print("=== Cannot generate map without valid coordinates or location data ===")
     map_image = None
else:
    GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY') # Load API key from secrets again for map generation
    if GOOGLE_API_KEY is None:
        print("Error: GOOGLE_API_KEY not found in Colab Secrets. Cannot generate map.")
        map_image = None
    else:
        map_image = generate_static_map(user_lat, user_lon, parks, bike_routes, GOOGLE_API_KEY)
        display(map_image)


In [ ]:
# Step 6: Generate Recommendations with LLM (Gemini API)

import google.generativeai as genai
from google.colab import userdata

# Configure Gemini API
try:
    GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')
    genai.configure(api_key=GOOGLE_API_KEY)
    print("Gemini API configured successfully.")
except Exception as e:
    print(f"Error configuring Gemini API: {e}")
    print("Please ensure your GOOGLE_API_KEY is set up in Colab Secrets.")

# Initialize the Generative Model
# Using a model suitable for text generation and potentially summarization
try:
    gemini_model = genai.GenerativeModel('gemini-1.5-flash-latest') # Or another suitable model like 'gemini-pro'
    print(f"Gemini model '{gemini_model.model_name}' initialized.")
except Exception as e:
    print(f"Error initializing Gemini model: {e}")
    gemini_model = None
    print("Could not initialize Gemini model. Recommendations will not be generated.")


def generate_llm_recommendations(weather, warnings, parks, bike_routes, aqi, category, recommendation, model, user_address):
    """Generate activity recommendations using a Gemini model."""
    print("=== Step 6: Generating LLM Recommendations ===")
    if not model:
        return "LLM model not available. Cannot generate recommendations."

    if not weather or not weather.get("main"):
        print("Warning: Missing weather data for LLM prompt.")
        weather_info = "weather data is unavailable."
    else:
        weather_info = (
            f"Current weather: {weather.get('main', 'Unknown').capitalize()}, "
            f"Temperature: {weather.get('temp', 'Unknown')}°C, "
            f"Wind speed: {weather.get('wind', 0.0):.1f} m/s."
        )

    # Perform replacement before inserting into f-string
    cleaned_recommendation = recommendation.replace('\n', '; ')
    air_quality_info = (
        f"Air Quality Index (AQI): {aqi} ({category}). "
        f"Health recommendation: {cleaned_recommendation}"
    )

    warnings_info = f"Active weather warnings: {', '.join(warnings) if warnings else 'None'}."

    nearby_locations_info = ""
    # Determine if weather is generally good for outdoors to include all locations
    is_good_weather = not warnings and category not in ["Moderate air quality", "Unhealthy for Sensitive Groups", "Unhealthy", "Very Unhealthy", "Hazardous"] and weather.get("main") not in ["Rain", "Thunderstorm", "Showers", "mist"] and not (weather.get("wind", 0) > 8 or weather.get("temp", 0) < 0 or weather.get("temp", 0) > 30)

    if parks or bike_routes:
        nearby_locations_info += "Nearby locations:\n"
        if parks:
            nearby_locations_info += "  Parks (suitable for walking):\n"
            # Include all parks if weather is good, otherwise limit to top 3
            parks_to_include = parks if is_good_weather else parks[:3]
            for park in parks_to_include:
                rating = f", Rating: {park.get('rating', 'N/A')}/5" if park.get("rating") != "N/A" else ""
                nearby_locations_info += f"    - {park['name']} ({park['distance']:.1f} km away){rating}\n"
        if bike_routes:
            nearby_locations_info += "  Bike Routes:\n"
            # Include all bike routes if weather is good, otherwise limit to top 3
            routes_to_include = bike_routes if is_good_weather else bike_routes[:3]
            for route in routes_to_include:
                rating = f", Rating: {route.get('rating', 'N/A')}/5" if route.get("rating") != "N/A" else ""
                nearby_locations_info += f"    - {route['name']} ({route['distance']:.1f} km away){rating}\n"
    else:
        nearby_locations_info = "No nearby parks or bike routes found."


    prompt = f"""
Given the following environmental conditions and nearby locations, provide a concise recommendation for outdoor activities near {user_address} today.

Conditions:
{weather_info}
{air_quality_info}
{warnings_info}

{nearby_locations_info}

Based on these conditions and locations, should I primarily consider indoor or outdoor activities? If outdoor activities are recommended and conditions are favorable, describe the suitability of the listed parks for walking and bike routes for biking, mentioning their distance and rating. If outdoor activities are not recommended, explain why and suggest indoor alternatives. Use a friendly and helpful tone.
"""
    print("Sending prompt to Gemini model...")
    try:
        response = model.generate_content(prompt)
        llm_recommendation = response.text.strip()
        print("Success: LLM recommendation generated.")
        print("LLM Output:")
        print(llm_recommendation)
        return llm_recommendation
    except Exception as e:
        print(f"Error generating content with Gemini model: {e}")
        return "Could not generate recommendation using LLM."


# Run immediately if conditions met
if weather is None or not weather.get("main"):
    print("=== Step 6: Error: No recommendations due to missing weather data. ===")
    final_recommendation_llm = "Error: Missing weather data."
elif user_lat is None or user_lon is None:
    print("=== Step 6: Error: No recommendations without valid coordinates. ===")
    final_recommendation_llm = "Error: Missing coordinates."
elif aqi is None or category is None or recommendation is None:
    print("=== Step 6: Error: No recommendations without air quality data. ===")
    final_recommendation_llm = "Error: Missing air quality data."
elif parks is None or bike_routes is None:
     print("=== Step 6: Error: No recommendations without parks and bike routes data. ===")
     final_recommendation_llm = "Error: Missing parks or bike routes data."
elif gemini_model is None:
    print("=== Step 6: Error: Gemini model not initialized. Cannot generate LLM recommendations. ===")
    final_recommendation_llm = "Error: LLM model not available."
else:
    # Pass USER_ADDRESS to the recommendation function
    final_recommendation_llm = generate_llm_recommendations(weather, warnings, parks, bike_routes, aqi, category, recommendation, gemini_model, USER_ADDRESS)
    print(f"\n=== Step 6b: Final Output (LLM) ===")
    weather_main = weather.get('main', 'Unknown').capitalize()
    weather_temp = f"{weather.get('temp', 'Unknown')}°C"
    weather_wind = f"{weather.get('wind', 0.0):.1f} m/s"
    print(f"Weather in {USER_ADDRESS}: {weather_main}, {weather_temp}, Wind: {weather_wind}")
    print(f"Air Quality Index: {aqi} ({category})")
    print(f"Weather Warnings: {', '.join(warnings) if warnings else 'None'}")
    print("LLM Recommendation:")
    print(final_recommendation_llm)

    # Generate and display the static map after the LLM output
    if 'generate_static_map' in globals():
        # Pass 'W' for parks and 'B' for bike routes labels
        map_image = generate_static_map(user_lat, user_lon, parks, bike_routes, GOOGLE_API_KEY)
        if map_image:
            display(map_image)
        else:
            print("=== Could not generate static map ===")
    else:
        print("=== Step 7: Error: generate_static_map function not found. Run def generate_static_map first. ===")

In [ ]:
# Step 8: Generate Email Message Content

# Ensure the final_recommendation_llm is available from Step 6 and weather data is available
if 'final_recommendation_llm' not in globals():
    print("=== Step 8: Error: Cannot generate email content. Please run Step 6 first. ===")
    print("Debug: final_recommendation_llm variable not found.")
elif 'weather' not in globals() or 'aqi' not in globals() or 'category' not in globals() or 'warnings' not in globals() or 'USER_ADDRESS' not in globals():
     print("=== Step 8: Error: Cannot generate email content. Missing weather or location data. Please run Steps 2-5 first. ===")
     print("Debug: weather, aqi, category, warnings, or USER_ADDRESS variable not found.")
else:
    email_title = "Today's Weather and Activity Recommendation"

    # Format the weather and air quality info
    weather_main = weather.get('main', 'Unknown').capitalize()
    weather_temp = f"{weather.get('temp', 'Unknown')}°C"
    weather_wind = f"{weather.get('wind', 0.0):.1f} m/s"
    # Construct as plain text lines for easier HTML formatting later
    weather_info_string = f"""
Current Conditions near {user_lat}, {user_lon}, {user_city}:

Weather: {weather_main}, {weather_temp}, Wind: {weather_wind}
Air Quality Index: {aqi} ({category})
Weather Warnings: {', '.join(warnings) if warnings else 'None'}

"""

    # Construct the email body as a plain text string
    cleaned_recommendation_lines = final_recommendation_llm.strip().split('\n')
    cleaned_recommendation_text = "\n".join([line.lstrip('*- ').replace('**', '') for line in cleaned_recommendation_lines]).strip()


    email_body_plain_text = f"""
{weather_info_string}

Recommendation:

{cleaned_recommendation_text}

---

Nearby Locations Map:

(Map Placeholder - The actual map image would be included here when sending the email)

"""

    # Display the generated email body as markdown (for preview in Colab)
    # Note: This display is just for preview. The actual email will be HTML.
    from IPython.display import display, Markdown
    display(Markdown(email_body_plain_text)) # Still display as markdown in Colab for readability

    # Display the static map image if it exists
    if 'map_image' in globals() and map_image is not None:
        display(map_image)
    else:
        print("=== Step 8b: Result: Static map variable not found or is None. Ensure Step 6 ran successfully. ===")

    # Store the plain text body in a new variable to be used in Step 9
    email_body_for_html = email_body_plain_text

In [ ]:
# Step 9 Sending email

import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
from email.mime.image import MIMEImage # Import MIMEImage
import ssl # For secure connection

# Ensure the email_body_for_html, email_title, and map_image are available
# Use email_body_for_html which is intended to be plain text from Step 8
if 'email_body_for_html' not in globals() or 'email_title' not in globals():
    print("Error: Required data (email_body_for_html, email_title) not found. Please run Steps 6 and 8 first.")
    # Set map_image to None if essential data is missing
    map_image = None
else:
    # Load GOOGLE_API_KEY for map generation if needed
    GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')
    if GOOGLE_API_KEY is None:
        print("Error: GOOGLE_API_KEY not found in Colab Secrets. Cannot generate map for email.")
        map_image = None # Ensure map_image is None if key is missing
    elif 'generate_static_map' in globals() and user_lat is not None and user_lon is not None and parks is not None and bike_routes is not None:
        # Regenerate map_image if the function exists and necessary data is available
        map_image = generate_static_map(user_lat, user_lon, parks, bike_routes, GOOGLE_API_KEY)
    else:
        print("Warning: Could not regenerate map for email. generate_static_map function or necessary data missing.")
        map_image = None # Ensure map_image is None if regeneration fails


    # Email configuration
    # Load sender email address and App Password/password from Colab Secrets
    # IMPORTANT: For security, use an App Password for services like Gmail.
    # You need to add 'SENDER_EMAIL' and 'SENDER_APP_PASSWORD' to your Colab Secrets.
    sender_email = userdata.get('SENDER_EMAIL')
    sender_password = userdata.get('SENDER_APP_PASSWORD') # Use App Password

    # Replace with recipient email (can be the same as sender)
    receiver_email = "xxxxxx@gmail.com" # Replace with recipient email

    # Check if sender credentials were loaded successfully
    if sender_email is None or sender_password is None:
        print("Error: Sender email or password not found in Colab Secrets.")
    else:
        # Create a secure SSL context
        context = ssl.create_default_context()

        try:
            # Connect to the SMTP server (example for Gmail)
            smtp_server = "smtp.gmail.com"
            port = 465  # For SSL

            print(f"Attempting to connect to SMTP server: {smtp_server} on port {port}...")
            with smtplib.SMTP_SSL(smtp_server, port, context=context) as server:
                print("Connection successful. Logging in...")
                server.login(sender_email, sender_password)
                print("Login successful. Sending email...")

                # Create the base message container.
                # We want to send an HTML email with an embedded image.
                msg = MIMEMultipart('related') # Use 'related' for embedding images

                msg['From'] = sender_email
                msg['To'] = receiver_email
                msg['Subject'] = email_title

                # --- Start HTML Conversion ---
                # Process the plain text body from Step 8 and convert to HTML structure
                plain_text_lines = email_body_for_html.strip().split('\n')
                html_lines = []
                in_weather_section = False
                in_recommendation_section = False
                in_map_section = False

                # Prepare the location string using lat, lon, city
                location_string = f"{user_lat}, {user_lon}, {user_city.capitalize()}"

                # Define a consistent style for section headers
                header_style = "font-weight: bold; font-size: 1.2em;" # Slightly larger and bold

                for line in plain_text_lines:
                    stripped_line = line.strip()

                    # Update the header line to use the new location string and inline style, wrap content in <strong>
                    if stripped_line == "Current Conditions near " + USER_ADDRESS + ":":
                        html_lines.append(f'<h2 style="{header_style}"><strong>Current Conditions near {location_string}:</strong></h2>')
                        in_weather_section = True
                        in_recommendation_section = False
                        in_map_section = False
                    elif stripped_line.startswith("Weather:"):
                         html_lines.append(f'<p><strong>Weather:</strong> {stripped_line[len("Weather:"):].strip()}</p>')
                    elif stripped_line.startswith("Air Quality Index:"):
                         html_lines.append(f'<p><strong>Air Quality Index:</strong> {stripped_line[len("Air Quality Index:"):].strip()}</p>')
                    elif stripped_line.startswith("Weather Warnings:"):
                         html_lines.append(f'<p><strong>Weather Warnings:</strong> {stripped_line[len("Weather Warnings:"):].strip()}</p>')
                    elif stripped_line == "Recommendation:":
                        # Apply style and wrap content in <strong>
                        html_lines.append(f'<h2 style="{header_style}"><strong>Recommendation:</strong></h2>')
                        in_weather_section = False
                        in_recommendation_section = True
                        in_map_section = False
                    elif stripped_line == "Nearby Locations Map:":
                        # Apply style and wrap content in <strong>
                         html_lines.append(f'<h2 style="{header_style}"><strong>Nearby Locations Map:</strong></h2>')
                         in_weather_section = False
                         in_recommendation_section = False
                         in_map_section = True
                    elif stripped_line == "---":
                         html_lines.append('<hr>') # Horizontal rule
                         in_weather_section = False
                         in_recommendation_section = False
                         in_map_section = False
                    elif stripped_line == "(Map Placeholder - The actual map image would be included here when sending the email)":
                        html_lines.append('IMAGE_PLACEHOLDER') # Placeholder for image
                    elif in_recommendation_section and stripped_line:
                        # Wrap recommendation text lines in paragraphs
                        html_lines.append(f'<p>{stripped_line}</p>')
                    elif stripped_line: # Default to paragraph for other non-empty lines
                        html_lines.append(f'<p>{stripped_line}</p>')


                html_body_content_processed = "\n".join(html_lines)

                # Add basic HTML structure and reference the image with a Content-ID
                image_cid = 'static_map_image' # Unique ID for the image

                html_body = f"""\
<html>
<head></head>
<body>
{html_body_content_processed.replace('IMAGE_PLACEHOLDER', f'<img src="cid:{image_cid}">')}
</body>
</html>
"""
                # --- End HTML Conversion ---

                # Attach the HTML part
                msg.attach(MIMEText(html_body, 'html'))

                # Attach the image if map_image is available
                if map_image is not None:
                    try:
                        img_url = map_image.url # Get the image URL from the IPython.display.Image object
                        img_data = requests.get(img_url, stream=True).content # Fetch image data
                        img = MIMEImage(img_data)
                        img.add_header('Content-ID', f'<{image_cid}>') # Add the Content-ID header
                        img.add_header('Content-Disposition', 'inline', filename='map.png') # Suggest filename
                        msg.attach(img)
                        print("Static map image attached successfully.")
                    except Exception as e:
                        print(f"Error fetching or attaching map image: {e}")
                        print("Sending email without embedded map.")
                        # If image attachment fails, send the HTML without the image tag
                        # Recreate the HTML body without the image placeholder
                        html_body_no_img = f"""\
<html>
<head></head>
<body>
{html_body_content_processed.replace('IMAGE_PLACEHOLDER', '<p>Could not load map image.</p>')}
</body>
</html>
"""
                        # Replace the related multipart with a simple HTML multipart if image fails
                        msg = MIMEMultipart('alternative')
                        msg['From'] = sender_email
                        msg['To'] = receiver_email
                        msg['Subject'] = email_title
                        msg.attach(MIMEText(html_body_no_img, 'html'))
                else:
                    print("No map image available to attach.")
                    # If map_image is None, send the HTML without the image tag
                    html_body_no_img = f"""\
<html>
<head></head>
<body>
{html_body_content_processed.replace('IMAGE_PLACEHOLDER', '<p>Map image not available.</p>')}
</body>
</html>
"""
                    # Replace the related multipart with a simple HTML multipart
                    msg = MIMEMultipart('alternative')
                    msg['From'] = sender_email
                    msg['To'] = receiver_email
                    msg['Subject'] = email_title
                    msg.attach(MIMEText(html_body_no_img, 'html'))


                server.sendmail(sender_email, receiver_email, msg.as_string())
                print("Email sent successfully!")

        except Exception as e:
            print(f"Error sending email: {e}")
            print("Please double-check your email address, password/App Password, SMTP server, and port.")
            print("If using Gmail, ensure you have enabled Less secure app access or used an App Password.")